## ***Responsible AI - 2024***
### *Proyecto #1*

Stefano Aragoni, Carol Arévalo, José Miguel González, Luis Santos

--------------------

Este proyecto se centra en la aplicación y el estudio de métodos de interpretación modelo-agnósticos, con un enfoque específico en SHAP (SHapley Additive exPlanations), para analizar y explicar las predicciones de churn de clientes en el sector de telecomunicaciones. Utilizando un dataset de churn de clientes de una compañía de telecomunicaciones disponible en Kaggle, el proyecto implementará modelos de predicción de churn, como árboles de decisión y Random Forest, y aplicará SHAP para desentrañar la influencia de diversas características en las predicciones del modelo.

### Objetivo del Proyecto

El objetivo principal de este proyecto es demostrar cómo los métodos de interpretación modelo-agnósticos pueden ser utilizados para proporcionar insights claros y comprensibles sobre las predicciones de modelos complejos de machine learning, específicamente en el contexto de la predicción de churn de clientes. Al hacerlo, el proyecto busca:

1. **Implementar y evaluar modelos predictivos** para la predicción de churn utilizando técnicas de machine learning como árboles de decisión y Random Forest.

2. **Aplicar SHAP**, un método de interpretación modelo-agnóstico, para analizar cómo diferentes características contribuyen a las predicciones de churn, permitiendo una interpretación detallada y justificable de los resultados del modelo.

3. **Visualizar y comunicar la importancia y el efecto de las características** en las decisiones del modelo, usando las visualizaciones proporcionadas por SHAP, como los plots de fuerza, de dependencia, y los summary plots.

4. **Facilitar la toma de decisiones basadas en datos** en el sector de telecomunicaciones, ayudando a identificar factores clave que influyen en el churn de clientes y posibles áreas de intervención para mejorar la retención de clientes.

Este enfoque no solo aumenta la transparencia de los modelos predictivos utilizados sino que también mejora la confianza de los stakeholders al proporcionar una comprensión clara de por qué ciertos clientes son más propensos a abandonar los servicios, apoyando así la creación de estrategias de retención más efectivas basadas en datos concretos y análisis robusto.

-------

#### *Setup*

Como primer paso, se importan las librerías necesarias para el desarrollo del proyecto. Asimismo, se realiza la configuración inicial requerida para el análisis. 

In [45]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelado
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [46]:
# Lectura de dataset
df = pd.read_csv('./dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [47]:
# Corrección del tipo de dato en columna 'TotalCharges'
    # Esta columna tiene valores numéricos. Sin embargo, Pandas la reconoce como object. Por lo tanto, se convierte a numérica.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') 

-------

#### *Exploratory Data Analysis (EDA)*

El segundo paso consiste en realizar un análisis exploratorio de los datos para comprender la estructura y las características del dataset. Esto incluye la identificación de las variables, verificación de valores faltantes, análisis gráfico de datos, entre otros. 

##### *Descripción de Columnas*

El primer paso en el análisis exploratorio de datos es comprender la estructura del dataset y las variables que contiene. 

- Se utiliza la función de pandas `info()` para obtener información sobre las columnas y los tipos de datos presentes en el dataset. 

- Se utiliza la función `head()` para visualizar las primeras filas del dataset y tener una idea de cómo están estructurados los datos.

- Se utiliza la función `describe()` para obtener estadísticas descriptivas de las variables presentes en el dataset.


La función `info()`, como se puede observar, indica que hay 7,043 registros en el dataset y 21 columnas. Además, se puede ver que hay variables categóricas (object) y numéricas (int64 y float64) presentes en el dataset.

Inicialmente, también se puede observar que no hay valores nulos en el dataset. Esto debido a que la columna Non-Null Count para todas las variables es igual al número total de registros en el dataset (7,043).

In [48]:
# Información general del dataset (columnas, valores nulos, tipo de datos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


La función `head()` permite visualizar las primeras filas del dataset. Como se puede observar, hay varias columnas que contienen información sobre los clientes, como el género, la edad, el estado civil, la educación, entre otros. Además, hay columnas que contienen información sobre los servicios contratados por los clientes. Finalmente, la columna **Churn** indica si el cliente ha abandonado o no los servicios de la compañía.

Con base a la información proporcionada por el dataset, se identificaron las siguientes columnas:

- **customerID**: Identificador único de cada cliente.  

- **gender**: Género del cliente. (Male/Female)

- **SeniorCitizen**: Indica si el cliente es una persona mayor. (1/0) 

- **Partner**: Indica si el cliente tiene pareja. (Yes/No)

- **Dependents**: Indica si el cliente tiene dependientes. (Yes/No)

- **tenure**: Número de meses que el cliente ha estado con la empresa.  

- **PhoneService**: Indica si el cliente tiene servicio telefónico. (Yes/No)

- **MultipleLines**: Indica si el cliente tiene múltiples líneas telefónicas. (Yes/No/No phone service)

- **InternetService**: Tipo de servicio de internet que el cliente tiene. (DSL/Fiber optic/No)

- **OnlineSecurity**: Indica si el cliente tiene servicio de seguridad en línea. (Yes/No)

- **OnlineBackup**: Indica si el cliente tiene servicio de respaldo en línea. (Yes/No)

- **DeviceProtection**: Indica si el cliente tiene protección de dispositivos. (Yes/No)

- **TechSupport**: Indica si el cliente tiene soporte técnico. (Yes/No)

- **StreamingTV**: Indica si el cliente tiene servicio de transmisión de TV. (Yes/No)

- **StreamingMovies**: Indica si el cliente tiene servicio de transmisión de películas. (Yes/No)

- **Contract**: Tipo de contrato del cliente. (Month-to-month/One year/Two year)

- **PaperlessBilling**: Indica si el cliente utiliza facturación electrónica. (Yes/No)

- **PaymentMethod**: Método de pago utilizado por el cliente. (Electronic check/Mailed check/Bank transfer (automatic)/Credit card (automatic))

- **MonthlyCharges**: Cantidad mensual que el cliente paga por los servicios.  

- **TotalCharges**: Cantidad total que el cliente ha pagado por los servicios desde que se unió.  

- **Churn**: Indica si el cliente ha dejado de utilizar los servicios. (Yes/No) (VARIABLE OBJETIVO)

In [49]:
# Visualización de los primeros registros
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#

Posteriormente, se utiliza la función `describe()` para obtener estadísticas descriptivas de las variables numéricas presentes en el dataset.

En este caso, el dataset original contiene 4 variables numéricas: **SeniorCitizen**, **tenure**, **MonthlyCharges** y **TotalCharges**.

- **SeniorCitizen**: Esta variable es binaria y toma valores 0 o 1. La media de esta variable es 0.162, lo que indica que aproximadamente el 16.2% de los clientes son personas mayores.

- **tenure**: Esta variable representa el número de meses que el cliente ha estado con la empresa. La media de esta variable es 32.4 meses, con un mínimo de 0 meses y un máximo de 72 meses.

- **MonthlyCharges**: Esta variable representa la cantidad mensual que el cliente paga por los servicios. La media de esta variable es 64.8, con un mínimo de 18.25 y un máximo de 118.75.

- **TotalCharges**: Esta variable representa la cantidad total que el cliente ha pagado por los servicios desde que se unió. La media de esta variable es 2283.3, con un mínimo de 18.8 y un máximo de 8684.8.

In [50]:
# Descripción de columnas numéricas
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


El procedimiento anterior se repitió pero para las variables categóricas presentes en el dataset.

A través de este análisis, se obtuvo una visión general de las variables presentes en el dataset y se identificaron las características clave. Por ejemplo:

- La mayoría de los clientes son personas hombres (3,555)
- La mayoría de los clientes no tienen dependientes (4,933)
- Una gran proporción de clientes tiene servicio telefónico (6,361)
- La mayoría de los clientes no son CHURN (5,174)

In [51]:
# Descripción de columnas categóricas
df.describe(include='object')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,7590-VHVEG,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174
